In [13]:
import numpy as np

In [14]:
def relu(input):
    return max(0, input)

In [15]:
# Multi- Layer Neural Network

import collections


def forwardPropagation(input, weights):
    number_of_layers = len(weights['hidden'])
    activations = {}
    
    for layer in range(number_of_layers):
        hidden_layer_size = len(weights['hidden'][layer])
        hidden_layer_values = np.zeros(hidden_layer_size)
        activations[layer] = {}
        for i in range(hidden_layer_size):
            hidden_layer_values[i] = np.dot(input, weights['hidden'][layer][i])
            hidden_layer_values[i] = relu(hidden_layer_values[i])
            activations[layer][i] = hidden_layer_values[i]
        
        input = hidden_layer_values
    # print(activations)
    output_values = np.dot(input,weights['output'])
    return output_values, activations


In [16]:
input_data = np.array([3,5])
weights = {
    'hidden': {0: [np.array([2, 4]), np.array([ 4, -5])], 
                1: [np.array([-1,  2]), np.array([1, 2])]},
    'output': np.array([2, 7])}

output, activations = forwardPropagation(input_data, weights)
print(output)
print(activations)

182.0
{0: {0: 26.0, 1: 0.0}, 1: {0: 0.0, 1: 26.0}}


In [52]:
def relu_derivative(x):
    # Derivative of ReLU function
    return np.where(x > 0, 1, 0)

In [101]:
def backPropagation(input, weights,target_value, learning_rate = 1):
    number_of_layers = len(weights['hidden'])
    
    predicted_value, activations = forwardPropagation(input, weights)
    
    print("Forward Propagation value: ", predicted_value)
    print("Activations", activations)
    
    delta_output_layer = np.subtract(predicted_value,target_value)
    deltas = {'output':delta_output_layer}
    
    # print("Delta output", deltas)
    #reverse track from last layer to the first
    deltas['hidden'] = {}
    for layer in range(number_of_layers - 1, -1, -1):
        # print("\n l: ", layer)
        # getting the layer weights and layer_activations
        hidden_layer_size = len(weights['hidden'][layer])
        layer_weights = weights['hidden'][layer]
        layer_activations = activations[layer]
    
        deltas['hidden'][layer] = {}
        # print("Deltas hidden", deltas )
        #computing deltas for hidden layers
        
        if layer == number_of_layers - 1:
            #layer before the output layer
            next_layer_weights = weights['output']
            next_layer_deltas = deltas['output']
            weighted_sum_next_layer = np.dot(next_layer_deltas, next_layer_weights)
        else:
            # other hidden layers
            next_layer_weights = weights['hidden'][layer + 1]
            # print("next weights", next_layer_weights)
            next_layer_deltas = deltas['hidden'][layer + 1]
            # print("next_deltas", next_layer_deltas)
            next_layer_deltas_array = np.array(list(next_layer_deltas.values()))
            # print("next_layer_deltas_array", next_layer_deltas_array)
            weighted_sum_next_layer = np.dot(next_layer_deltas_array, next_layer_weights)
            # print("weighted_sum_next_layer", weighted_sum_next_layer)
        
        for i in range(hidden_layer_size):
            # print("layer activations", layer_activations[i])
            # print("weighted_sum_next_layer", weighted_sum_next_layer[i])
            deltas['hidden'][layer][i] = np.dot(weighted_sum_next_layer[i],layer_activations[i])
    
    return deltas

In [122]:
input_data = np.array([2,3])
weights = {
    'hidden': {0: [np.array([0.5, 0.4]), np.array([0.3, -0.2])], 
                1: [np.array([0.6, -0.3]), np.array([0.4, 0.8])]},
    'output': np.array([1, -1])}
target_value = 1

In [133]:
def update_weights(weights, activations, deltas, input, learning_rate):
    # Update hidden layers' weights
    for layer in range(len(weights['hidden'])):
        layer_activations = np.array(list(activations[layer].values()))
        print("layer", layer, "layer_activations", layer_activations)
        
        if layer == (len(weights['hidden']) - 1):
            next_layer_weights = weights['output']
        else:
            next_layer_weights = weights['hidden'][layer + 1]
            print("next_layer_weights", next_layer_weights)
    # Update output layer's weights
    
    
    

In [134]:
def totalBP(input_data, weights):
    deltas = backPropagation(input_data, weights,target_value, learning_rate = 1)
    update_weights(weights, activations, deltas, input_data, 1)
    return weights


w = totalBP(input_data, weights)

print("w", w)
    

Forward Propagation value:  0.43999999999999995
Activations {0: {0: 2.2, 1: 0.0}, 1: {0: 1.32, 1: 0.8800000000000001}}
layer 0 layer_activations [2.2 0. ]
next_layer_weights [array([ 0.6, -0.3]), array([0.4, 0.8])]
layer 1 layer_activations [1.32 0.88]
w {'hidden': {0: [array([0.5, 0.4]), array([ 0.3, -0.2])], 1: [array([ 0.6, -0.3]), array([0.4, 0.8])]}, 'output': array([ 1, -1])}
